# CarND-Behavioral-Cloning-P3

In [ ]:
import csv
import os
import cv2
import numpy as np
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline 

BATCH_SIZE=128

In [ ]:
SAVED_DIM_TRACKS_DIR='../saved_sim_tracks'

samples=[]
def prepare_data():
    with open(os.path.join(SAVED_DIM_TRACKS_DIR, 'driving_log.csv')) as csvFile:
        reader = csv.reader(csvFile)
        for line in reader:
            samples.append(line)

prepare_data()            
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

print('train_samples: ', len(train_samples))
print('validation_samples: ', len(validation_samples))

In [ ]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                for i in range(0,3):
                    filename = batch_sample[0].split('/')[-1]
                    name = os.path.join(SAVED_DIM_TRACKS_DIR, 'IMG', filename)
                    img = cv2.imread(name)
                    angle = float(batch_sample[3])
                    
                    if (i==1):
                        angle+=0.2
                    elif (i==2):
                        angle-=0.2
                        
                    images.append(img)
                    angles.append(angle)
                    
                    # apply our augmentation to the original images
                    flipped_img=cv2.flip(img,1)
                    flipped_angle=angle*-1 
                    images.append(flipped_img)
                    angles.append(flipped_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=BATCH_SIZE)
validation_generator = generator(validation_samples, batch_size=BATCH_SIZE)

## Models

In [ ]:
#-------------------------- set gpu using tf ---------------------------
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

#-------------------  start importing keras module ---------------------
import keras.backend.tensorflow_backend as K
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint

K._get_available_gpus()

### NVidia

In [ ]:
model=Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(160,320,3)))
# model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
# crop it
model.add(Cropping2D(cropping=((50,20),(0,0))))

##### NVidia
model.add(Conv2D(24,(5,5),strides=(2,2),activation='relu'))
model.add(Conv2D(36,(5,5),strides=(2,2),activation='relu'))
model.add(Conv2D(48,(5,5),strides=(2,2),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.summary()

model.compile(loss='mse', optimizer='adam')
#model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

save_to='model-nvidia.h5'

early_stopper=EarlyStopping(patience=5, verbose=1)
checkpointer=ModelCheckpoint(save_to, monitor='val_loss', verbose=1, save_best_only=True)

history_object = model.fit_generator(generator=train_generator, 
                                     steps_per_epoch=len(train_samples) / BATCH_SIZE, 
                                     validation_data=validation_generator,
                                     validation_steps=len(validation_samples) / BATCH_SIZE,
                                     callbacks=[checkpointer, early_stopper],
                                     use_multiprocessing=True,
                                     epochs=3, 
                                     verbose=2)

model.save(save_to)

### LeNet

In [ ]:
model=Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(160,320,3)))
# crop it
model.add(Cropping2D(cropping=((50,20),(0,0))))

##### LeNet
model.add(Conv2D(10,5,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(50,5,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.summary()

model.compile(loss='mse', optimizer='adam')

save_to='model-lenet.h5'

early_stopper=EarlyStopping(patience=5, verbose=1)
checkpointer=ModelCheckpoint(save_to, monitor='val_loss', verbose=1, save_best_only=True)

history_object = model.fit_generator(generator=train_generator, 
                                     steps_per_epoch=len(train_samples), 
                                     validation_data=validation_generator,
                                     validation_steps=len(validation_samples), 
                                     epochs=1, verbose=1)
model.save(save_to)

## View Results

In [ ]:
### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()